In [ ]:
import pandas as pd
import numpy as np
import os
import datetime
from datetime import datetime, date, time, timezone
from datetime import timedelta  
import matplotlib.pyplot as plt
import string
from opt_functions import SusRes_MBC

In [ ]:
dir = '/Data_logs/outputs_n_GPUs/vgg16/batch_512/4gpu/'

In [ ]:
Pidle_server_with_storage = 75 + 2*30 + 24*3 + 5
Pmax_server_with_storage = 75 + 2*30 + 36*7 + 24*8 + 18 #or 537 withought 2*30!!

Pidle_server = 75 + 2*30 + 24*3
Pmax_server = 75 + 2*30 + 36*7 + 24*8

In [ ]:
def reading_gpu_data(job, batch, n_gpu, per):
    
    job_power_dir = dir + job +'/batch_' + str(batch) + '/' + str(n_gpu) + 'gpu/'
    
    job_power_data_gpu = pd.read_csv(job_power_dir + 'gpu_nvidia' +'.csv')
    job_power_data_gpu.columns = [c.replace(' ', '_') for c in job_power_data_gpu.columns]
    job_power_data_gpu.rename(columns={"_power.draw_[W]": "power", "_timestamp":"time", "_utilization.gpu_[%]":"gpu_util"
                                  , "_temperature.gpu":"gpu_temp", "":""}, inplace=True)
    
    job_power_data_gpu.drop(job_power_data_gpu.columns.difference(['index','power','time', 'gpu_util', 'gpu_temp', 'memory_used(MiB)', 'memory_total(MiB)']), 1, inplace=True)
    
    job_power_data_gpu.power = job_power_data_gpu.power.str.replace('[^\d.]', '').astype(float)
    job_power_data_gpu.gpu_util = job_power_data_gpu.gpu_util.str.replace('[^\d.]', '').astype(float)
    
    job_power_data_gpu['time'] = job_power_data_gpu['time'].str.replace('\.\d\d\d', '')
    job_power_data_gpu.time = pd.to_datetime(job_power_data_gpu.time, format='%Y-%m-%d %H:%M:%S')
    
    GPU0_data = job_power_data_gpu[job_power_data_gpu['index']==0].copy()
    GPU1_data = job_power_data_gpu[job_power_data_gpu['index']==1].copy()
    GPU2_data = job_power_data_gpu[job_power_data_gpu['index']==2].copy()
    GPU3_data = job_power_data_gpu[job_power_data_gpu['index']==3].copy()
    GPU4_data = job_power_data_gpu[job_power_data_gpu['index']==4].copy()
    GPU5_data = job_power_data_gpu[job_power_data_gpu['index']==5].copy()
    GPU6_data = job_power_data_gpu[job_power_data_gpu['index']==6].copy()
    GPU7_data = job_power_data_gpu[job_power_data_gpu['index']==7].copy()
    
    GPU0_data = GPU0_data.set_index('time').resample(per).mean()
    GPU1_data = GPU1_data.set_index('time').resample(per).mean()
    GPU2_data = GPU2_data.set_index('time').resample(per).mean()
    GPU3_data = GPU3_data.set_index('time').resample(per).mean()
    GPU4_data = GPU4_data.set_index('time').resample(per).mean()
    GPU5_data = GPU5_data.set_index('time').resample(per).mean()
    GPU6_data = GPU6_data.set_index('time').resample(per).mean()
    GPU7_data = GPU7_data.set_index('time').resample(per).mean()
    
    return(GPU0_data, GPU1_data, GPU2_data, GPU3_data, GPU4_data, GPU5_data, GPU6_data, GPU7_data)

In [ ]:
def reading_cpu_data(job, batch, n_gpu, ex_date, per):
    
    job_power_dir = dir + job +'/batch_' + str(batch) + '/' + str(n_gpu) + 'gpu/'
   
    job_power_data_cpu_top = pd.read_csv(job_power_dir + 'cpu_top_fixed' +'.csv')

    job_power_data_cpu_top.drop(job_power_data_cpu_top.columns.difference(['time','cpu_us', 'cpu_sy']), 1, inplace=True)
    
    #********************************************************************
    d1 = ex_date
    d2 = date(1900,1,1)
    result1 = abs(d2-d1).days
    #********************************************************************
    
    job_power_data_cpu_top.time = pd.to_datetime(job_power_data_cpu_top.time, format='%H:%M:%S') + timedelta(result1) 
    job_power_data_cpu_top = job_power_data_cpu_top.set_index('time').resample(per).mean()
    
    job_power_data_cpu_top['cpu_util'] = job_power_data_cpu_top['cpu_us'] + job_power_data_cpu_top['cpu_sy']
    
    return (job_power_data_cpu_top)

In [ ]:
def calculating_power(job, batch, n_gpu, ex_date, per, storage_used):
    
    power_df = reading_cpu_data(job, batch, n_gpu, ex_date, per)
    power_df.drop(columns=['cpu_us', 'cpu_sy'], inplace=True)

    g0, g1, g2, g3, g4, g5, g6, g7 = reading_gpu_data(job, batch, n_gpu, per)
    
    date0 = g0.index[0]
    date1 = g0.index[-1]
    

    power_df = power_df.loc[date0:date1].copy()
    
    if storage_used==True:
        power_df['server_power'] = Pidle_server_with_storage + (Pmax_server_with_storage-Pidle_server_with_storage)*(power_df['cpu_util']/100)
    else:
        power_df['server_power'] = Pidle_server + (Pmax_server-Pidle_server)*(power_df['cpu_util']/100)
    
    power_df['gpu_power'] = 0
    power_df['gpu_util'] = 0
    
    
    if n_gpu==1:
        power_df['gpu_power'] = g0['power']
        power_df['gpu_util'] = g0['gpu_util']
    elif n_gpu==2:
        power_df['gpu_power'] = g0['power'] + g1['power']
        power_df['avg_gpu_util'] = (g0['gpu_util'] + g1['gpu_util'])/2
    elif n_gpu==3:
        power_df['gpu_power'] = g0['power'] + g1['power'] + g2['power']
        power_df['avg_gpu_util'] = (g0['gpu_util'] + g1['gpu_util'] + g2['gpu_util'])/3
    elif n_gpu==4:
        power_df['gpu_power'] = g0['power'] + g1['power'] + g2['power'] + g3['power']
        power_df['avg_gpu_util'] = (g0['gpu_util'] + g1['gpu_util'] + g2['gpu_util'] + g3['gpu_util'])/4
    elif n_gpu==5:
        power_df['gpu_power'] = g0['power'] + g1['power'] + g2['power'] + g3['power'] + g4['power']
        power_df['avg_gpu_util'] = (g0['gpu_util'] + g1['gpu_util'] + g2['gpu_util'] + g3['gpu_util'] + g4['gpu_util'])/5
    elif n_gpu==6:
        power_df['gpu_power'] = g0['power'] + g1['power'] + g2['power'] + g3['power'] + g4['power'] + g5['power']
        power_df['avg_gpu_util'] = (g0['gpu_util'] + g1['gpu_util'] + g2['gpu_util'] + g3['gpu_util'] + g4['gpu_util'] + g5['gpu_util'])/6
    elif n_gpu==7:
        power_df['gpu_power'] = g0['power'] + g1['power'] + g2['power'] + g3['power'] + g4['power'] + g5['power'] + g6['power']
        power_df['avg_gpu_util'] = (g0['gpu_util'] + g1['gpu_util'] + g2['gpu_util'] + g3['gpu_util'] + g4['gpu_util'] + g5['gpu_util'] + g6['gpu_util'])/7
    elif n_gpu==8:
        power_df['gpu_power'] = g0['power'] + g1['power'] + g2['power'] + g3['power'] + g4['power'] + g5['power'] + g6['power'] + g7['power']
        power_df['avg_gpu_util'] = (g0['gpu_util'] + g1['gpu_util'] + g2['gpu_util'] + g3['gpu_util'] + g4['gpu_util'] + g5['gpu_util'] + g6['gpu_util'] + g7['gpu_util'])/8

    power_df['tot_power'] = power_df['server_power'] + power_df['gpu_power']
    
    
    return (power_df)

In [ ]:
def avg_time_per_epoch(job, batch, n_gpu, up_time):
    
    g0, g1, g2, g3, g4, g5, g6, g7 = reading_gpu_data(job, batch, n_gpu, period)
    
    date0 = g0.index[0]
    date1 = g0.index[-1]
    
    avg_time_epoch = abs(date1-date0) - timedelta(seconds=up_time)
    
    return (avg_time_epoch)

In [ ]:
def estimated_total_time(job, batch, n_gpu, up_time, num_epochs):
    
    avg_time_epoch = avg_time_per_epoch(job, batch, n_gpu, up_time)
    jct = timedelta(seconds=up_time) + num_epochs*avg_time_epoch
    
    return (jct)

In [ ]:
def estimated_total_energy(job, batch, n_gpu, ex_date, per, up_time, num_epochs, storage_used):
    
    power_df = calculating_power(job, batch, n_gpu, ex_date, per, storage_used)
    average_tot_power = power_df.tot_power.mean() #watt
    
    total_time = estimated_total_time(job, batch, n_gpu, up_time, num_epochs)
    runtime_hour = total_time.total_seconds()/3600.0 #hour, H
    
    tot_energy = (average_tot_power * runtime_hour)/1000 #KwH
    
    return (tot_energy)

# VGG16

In [ ]:
up_time = [15, 25, 20, 25, 25, 35, 30]

x = np.zeros(8)
for i in range(2, 8):
    #print(estimated_total_energy('vgg16', 512, i, date(2022,8,4), period, up_time[i-1], 90, True))
    #print(avg_time_per_epoch('vgg16', 512, i, up_time[i-1]))
     print(estimated_total_time('vgg16', 512, i, up_time[i-1], 90)) 

In [ ]:
 print(estimated_total_energy('vgg16', 512, 8, date(2022,8,2), period, 30, 90, True))

In [ ]:
 print(avg_time_per_epoch('vgg16', 512, 8, 30))

In [ ]:
 print(estimated_total_time('vgg16', 512, 8, 30, 90))

# Plots

In [ ]:
up_time = [15, 25, 20, 25, 25, 35, 30]

energy = np.zeros(8)
jct = np.zeros(8)
avg_epoch = np.zeros(8)

for i in range(2, 8):
    energy[i-1] = estimated_total_energy('vgg16', 512, i, date(2022,8,4), period, up_time[i-1], 90, True)
    avg_epoch[i-1] = avg_time_per_epoch('vgg16', 512, i, up_time[i-1]).total_seconds()
    jct[i-1] = estimated_total_time('vgg16', 512, i, up_time[i-1], 90).total_seconds()
        
energy[7] = estimated_total_energy('vgg16', 512, 8, date(2022,8,2), period, 30, 90, True)
avg_epoch[7] = avg_time_per_epoch('vgg16', 512, 8, 30).total_seconds()
jct[7] = estimated_total_time('vgg16', 512, 8, 30, 90).total_seconds()       

In [ ]:
plt.plot(energy[1:8])